



Project Ttile: Real-Time Crypto Market Q&A Assistant"

In [1]:
pip install transformers accelerate bitsandbytes #install required packages --> bitsandbytes used for quantization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

- install hugging face tarnsformers - to access the Phi-3 LLm mode + tokenizer
- bitsandbytes - for quantization of the Phi-3 model - reduce memory usage
- faiss-cpu - facebook AI Similarity Search - Fast vector search on CPU
- sentence-transformers - convert text into vector embedding - hugging face ('all-MiniLM-L6-v2')
- torch - PyTorch - deep learning framework
- requests - HHTP Requests - to make web requests (fetch the data from CoinGecko API)

In [3]:
!pip install transformers accelerate bitsandbytes faiss-cpu sentence-transformers torch requests

In [23]:
import requests #library used to send HHTP request to CoinGecko API

def fetch_crypto_data():
    url = 'https://api.coingecko.com/api/v3/coins/markets' #CoinGecko API
    params = {
        'vs_currency': 'usd', #prices in usd
        'order': 'market_cap_desc', #sort market cap in descending order
        'per_page': 10, #get the top 10 coins
        'page': 1,
        'sparkline': False
    }
    response = requests.get(url, params=params) #api get rewuest
    data = response.json() #parse data

    docs = []
    for coin in data: #loop over each coin in the data --> format
        text = (
            f"Name: {coin['name']}\n"
            f"Symbol: {coin['symbol']}\n"
            f"Price: ${coin['current_price']}\n"
            f"Market Cap: ${coin['market_cap']}\n"
            f"24h Change: {coin['price_change_percentage_24h']}%\n"
        )
        docs.append(text) #append docs list -- > created above
    return docs

docs = fetch_crypto_data() #calling the function
print(docs[0]) #print first coin in the list -- show how the data is now formatted



Name: Bitcoin
Symbol: btc
Price: $101770
Market Cap: $2022722790068
24h Change: -0.95508%



In [41]:
print('Fetched:',len(docs),'cryptos') #confirm there is 10 coins
print(docs[0])

Fetched: 10 cryptos
Name: Bitcoin
Symbol: btc
Price: $101770
Market Cap: $2022722790068
24h Change: -0.95508%



In [25]:
from sentence_transformers import SentenceTransformer #embedding model -- text to numeric vectors
import faiss #facebook AI Similarity Search
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2') #lightweight embedding model
embeddings = embedder.encode(docs, convert_to_numpy=True) # converts text in docs into vector medding ---> return numpy array

index = faiss.IndexFlatL2(embeddings.shape[1]) #create FLATL2 index - euclidean disatnce search - vectorstore
index.add(embeddings) #add doc embedding into the FAISS index

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline #import tokenizer, model and pipeline helper
import torch

model_id = "microsoft/phi-3-mini-4k-instruct" #model used from hugging face

tokenizer = AutoTokenizer.from_pretrained(model_id) #load tokenizer from Phi-3 -- to tokenize the text
model = AutoModelForCausalLM.from_pretrained(model_id, #load Causal LM model weights
                                             torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, #IMPORTANT: use float16 preciison if GPU available (sometimes Google colab resources are limited)
                                             device_map="auto")

#Create the pipleine for text generation task
rag_pipeline = pipeline("text-generation",
                        model=model, #created above mdoel
                        tokenizer=tokenizer, #created above tokenizer
                        device_map="auto", #uses best available hardware (depending whats avilable on google colab -- resources can be limited)
                        return_full_text=False, #only return generated text -- not prompt
                        pad_token_id = tokenizer.eos_token_id, #used to avoid padding errors during inference
                        eos_token_id=tokenizer.eos_token_id) # sets end-of-sequence -- tells mdoel to stop generating


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [37]:
def retrieve_context(query, k=3): #k=3 -> k-nearest neightbor search on FAISS index
    query_embedding = embedder.encode([query], convert_to_numpy=True).astype("float32") #use embedding model imported earlier to convert text to numerical embedding vector - numpy array
    D, I = index.search(np.array(query_embedding), k) #semantic search seahc on FAISS index
    return "\n\n".join([docs[i] for i in I[0] if i != -1]) #combine docs return into a single string - sep by blank lines -- structured for prompts NOTE: -1 is a safety check - -1 can appear if FAISS doesnt find enough neightbours

def generate_answer(query): #takes query - generate RAG pipleline answer
    context = retrieve_context(query, k=3) #calling function above and returns top 3 relevent strings from vector store

#construct prompt - to send to llm -- > instructional-style (context-query)
    prompt = f"""
You are a crypto assistant. Use the context to answer the question concisely.

Context:
{context}

Question:
{query}

Answer:
"""

    response = rag_pipeline(
        prompt,
        max_new_tokens=60, #limit reponse length
        do_sample=False #disable sampling -- les randomness (no temperature required)
    )[0]["generated_text"].strip() #strip any whitespaces from models generated text


    answer = response.split("\n\n")[0].split('Question:')[0].strip() #format a generated reponse -- I was getting all tyes of  reponses with loads of Questions and answers so I had to add this piece of code to stop the model over generating an adding laods of Q&A's
    return answer

In [39]:
#list of question passed to LLm Model
questions = [
    "What is the market cap of Bitcoin?",
    "Which cryptocurrency has the highest 24h change?",
    "What is the price of XRP?"
]

for q in questions:  #iterate through the questions in the list above
    answer = generate_answer(q) #call RAG powered LLM function created above
    print(f"Question: {q}")
    print(f"Answer: {answer}\n")


#Note you can ignore the prompt below -- ['temperature] -- I have add do_sample=False

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What is the market cap of Bitcoin?
Answer: The market cap of Bitcoin is $2022722790068.



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: Which cryptocurrency has the highest 24h change?
Answer: Bitcoin has the highest 24h change at -0.95508%.

Question: What is the price of XRP?
Answer: The price of XRP is $2.03.

